## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [5]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
# file_index='BBBSBBBB'
file_index='BSBSBSBS'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1580
+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+-----------+------+--------------------+------+
|            coeff_1|            coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+-----------+------+--------------------+------+
|-357.31641050459905|-129.13504460482747| 75.15483323926503|      2.7|BSBSBSBS| 25.5011|   -80.55|       TOBS| 0.728934692929237|0.6935303738929723|0.6815386438589387| 0.02869278388895

In [6]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [45]:
#extract longitude and latitude for each station
feature='coeff_4'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_4 FROM weather


AnalysisException: u"cannot resolve '`coeff_4`' given input columns: [res_mean, total_var, coeff_2, latitude, year, coeff_1, label, longitude, coeff_3, elevation, station, res_1, res_2, res_3, measurement, vector, undefs]; line 1 pos 22;\n'Project [station#12, year#16, 'coeff_4]\n+- SubqueryAlias weather, `weather`\n   +- Relation[coeff_1#0,coeff_2#1,coeff_3#2,elevation#3,label#4,latitude#5,longitude#6,measurement#7,res_1#8,res_2#9,res_3#10,res_mean#11,station#12,total_var#13,undefs#14L,vector#15,year#16] parquet\n"

In [43]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_4')
year_station_table.head(10)

station,USC00080488,USC00080611,USC00080737,USC00080739,USC00080992,USC00081276,USC00081305,USC00081306,USC00081651,USC00081654,...,USC00087758,USC00087760,USC00088368,USC00088620,USC00088780,USC00088841,USC00089219,USC00089511,USC00089520,USW00092811
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [44]:
def getFraction(a,b):
    return (1-(a/b))*100
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))
def MS(Mat):
    return np.nanmean(Mat**2);

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS = %.2f'%MS(year_station_table)
print 'RMS removing mean-by-station = %.2f, fraction explained = %.2f'%(MS(tbl_minus_station),getFraction(MS(tbl_minus_station),MS(year_station_table)))
print 'RMS removing mean-by-year    = %.2f, fraction explained = %.2f'%(MS(tbl_minus_year),getFraction(MS(tbl_minus_year),MS(year_station_table)))

total MS = 234951.90
RMS removing mean-by-station = 92102.83, fraction explained = 60.80
RMS removing mean-by-year    = 194259.87, fraction explained = 17.32


In [14]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 484.718370945
0 after removing mean by year    = 440.749211132
0 after removing mean by stations= 273.57506377
1 after removing mean by year    = 267.882265232
1 after removing mean by stations= 267.322373796
2 after removing mean by year    = 267.223772838
2 after removing mean by stations= 267.194870316
3 after removing mean by year    = 267.182334023
3 after removing mean by stations= 267.175719944
4 after removing mean by year    = 267.171961878
4 after removing mean by stations= 267.169771761
